In [1]:
from src.constants import p, a, b
from src.CurvePoint import CurvePoint
from src.EllipticCurve import EllipticCurve


from src.Emitter import Emitter
from src.Receiver import Receiver

from src.utils import initPublic

In [2]:
P = CurvePoint(a, b, 8, 1)
public = initPublic(P)

In [3]:
A = Emitter("Python >> Ocaml", public)
B = Receiver(public)

In [5]:
A.encodeMsg(public)

In [6]:
B.decodeMsg(public)

'Python >> Ocaml'

In [7]:
public

{'stringToPoints': <function src.utils.createMaps.<locals>.stringToPoints(str)>,
 'pointsToString': <function src.utils.createMaps.<locals>.pointsToString(arr)>,
 'P': <src.CurvePoint.CurvePoint at 0x1b8d1401e10>,
 'q': 290,
 'A': array([[-1,  5, -1],
        [-2, 11,  7],
        [ 1, -5,  2]]),
 'aP': <src.CurvePoint.CurvePoint at 0x1b8d1403670>,
 'bP': (<src.CurvePoint.CurvePoint at 0x1b8d365b8b0>,
  array([<src.CurvePoint.CurvePoint object at 0x000001B8D365A290>,
        dtype=object))}